# Communication carte micro:bit ordinateur

L'objectif de cette partie est de commander la carte micro:bit depuis l'ordinateur

                                    port série/USB     python          
                       microbit  <------------------>    PC  

Le port est le point d'entrée /sortie de l'ordinateur
 

## Les 4 étapes de la communication entre la carte et l'ordinateur

Il y **toujours** 4 étapes à respecter pour communiquer entre la carte micro:bit et l'ordinateur 


1. trouver le port sur lequel est branché la carte micro:bit (ce sera toujours le même code)
2.  installer le programme sur la carte micro:bit
3.  mise en place de la communication entre l'ordinateur et la carte micro:bit
4.  clore la communication avec la carte micro:bit (ce sera toujours le même code)

##  Pour voir sur quel port  du PC est connecté la microbit

Selon les PC, votre carte sera connectée sur le port série 'COM3' ou le port série 'COM7' ou le port série 'COM123'. Bref, il faut voir quel est le port série sur lequel se trouve la microbit. En exécutant le code ci-dessous on affiche toutes les infos sur les ports série disponibles / ouverts.  
Sur mon PC, un seul en USB : donc c'était lui ('COM7') sur lequel était la microbit.

Il faut exécuter le programme suivant (cliquer sur la cellule puis "executer")

In [28]:
import serial.tools.list_ports as sr_tl

def scanner_ports_serie_pour_trouver_microbit():
    K = sr_tl.comports()
    for portInfo in K:
        print('----------------------')
        print(portInfo.device)
        print(portInfo.name)
        print(portInfo.description)
        print(portInfo.hwid)

scanner_ports_serie_pour_trouver_microbit()

----------------------
COM3
COM3
Périphérique série USB (COM3)
USB VID:PID=0D28:0204 SER=9901000050784E45000F10100000001F0000000097969901 LOCATION=1-1:x.1


## Pour clore la communication avec le port série 

Il est nécessaire de clore la communication proprement, sinon le port série est "immobilisé" et on ne peut plus s'y reconnnecter. Il faut alors débrancher la carte micro:bit du PC et la rebrancher.

Il faut exécuter le programme suivant (cliquer sur la cellule puis "executer")

In [ ]:
continuer_lecture = False                # on coupe la boucle puis la comm 
comm_serie.close()                       # sinon readline sur une comm coupée --> erreur

## Activité 1 :utilisation des capteurs interne de la  micro:bit


###    Programme à installer  sur la carte micro:bit 

**Rappels**

- copier-coller le code python dans l'éditeur [python pour microbit](https://python.microbit.org/v/2.0 " https://python.microbit.org/v/2.0 ")
- Dans l'éditeur python pour microbit, cliquer sur "Download"
- Glisser-Déposer le fichier microbit_program.hex téléchargé sur la carte microbit.

```python
 
import microbit

microbit.uart.init(baudrate=115200, bits=8, parity=None, stop=1)

while microbit.running_time() < 3600000:
    
    donnees = microbit.uart.readline()
    if donnees != None:
        message_recu = str(donnees, 'utf-8')
        microbit.display.show(message_recu)
        
    if microbit.button_a.is_pressed():
        microbit.uart.write('bouton A')
        microbit.sleep(1000)
        
    if microbit.button_b.is_pressed():
        microbit.uart.write('bouton B')
        microbit.sleep(1000)

```

###  Mise en place de la communication entre la carte micro:bit et l'ordinateur

On crée une "communication" serie que l'on écoute et sur laquelle on peut envoyer des messages. Cela permet de mettre en place la communication et recevoir les messages en provenance de la carte microbit

* Modifier la valeur du port dans le programme ci-dessous en fonction du code lancé au début : ce n'est pas forcément COM7
* Cliquer sur la cellule puis "executer" pour lancer le programme
* Ne pas oublier de clore


In [ ]:
import serial
from threading import Thread

comm_serie = serial.Serial('COM3', baudrate = 115200, bytesize = serial.EIGHTBITS, 
                           parity = serial.PARITY_NONE, stopbits = serial.STOPBITS_ONE, 
                           timeout=.1, write_timeout = 0)

# Modifier la valeur du port en fonction du code lancé au début : ce n'est pas forcément COM7


def ecoute(comm):
    while continuer_lecture:
        data = comm.readline().decode("utf-8")
        if data != None and len(data)!= 0:
            print(data)

def envoyer_message(message, comm):
    comm.write(message.encode("utf-8"))



continuer_lecture = True
thread_ecoute = Thread(target = lambda : ecoute(comm_serie))
thread_ecoute.start()

### Questions

1. Appuyer sur le bouton A , puis sur le bouton B, noter ce qui se passe

2. En déduire ce que fait la commande `microbit.uart.write('bouton B')` dans le programme téléchargé sur la carte micro:bit

3. Executer le code suivant (cliquer sur la cellule, puis "executer") et indiquer ce qui se passe. Modifier le contenu du message. 

In [3]:
message = ('HH ee ll ll oo !!')
envoyer_message(message, comm_serie)

## Activité 2 : commander l'éclairage

Il faut d'abord brancher le capteur de lumière sur le pin0 et la led rouge sur le pin1

###  Programme à installer  sur la carte micro:bit 



- copier-coller le code python dans l'éditeur [python pour microbit](https://python.microbit.org/v/2.0 " https://python.microbit.org/v/2.0 ")
- Dans l'éditeur python pour microbit, cliquer sur "Download"
- Glisser-Déposer le fichier microbit_program.hex téléchargé sur la carte microbit.

```python
import microbit

microbit.uart.init(baudrate=115200, bits=8, parity=None, stop=1)

while microbit.running_time() < 36000000: #10 heures
    
    donnees = microbit.uart.readline()
    if donnees != None:
        message_recu = str(donnees, 'utf-8')
        if message_recu == 'allume':
            microbit.pin1.write_digital(1)
        elif message_recu == 'eteint':
            microbit.pin1.write_digital(0)  
                  
    lum = microbit.pin0.read_analog()
    lum = str(lum)
    microbit.uart.write(lum)
    microbit.sleep(500)


```

### Mise en place de la communication entre la carte micro:bit et l'ordinateur

C'est le même programme que dans l'activité 2 (penser à modifier la valeur du port si nécessaire).

In [ ]:
import serial
from threading import Thread

comm_serie = serial.Serial('COM3', baudrate = 115200, bytesize = serial.EIGHTBITS, 
                           parity = serial.PARITY_NONE, stopbits = serial.STOPBITS_ONE, 
                           timeout=.1, write_timeout = 0)

# Modifier la valeur du port en fonction du code lancé au début : ce n'est pas forcément COM7


def ecoute(comm):
    while continuer_lecture:
        data = comm.readline().decode("utf-8")
        if data != None and len(data)!= 0:
            print(data)

def envoyer_message(message, comm):
    comm.write(message.encode("utf-8"))



continuer_lecture = True
thread_ecoute = Thread(target = lambda : ecoute(comm_serie))
thread_ecoute.start()

### Questions

1. Executer le programme avec différents éclairement (lumière forte, obscutité). Noter ce qui se passe 

2. Executer le code suivant (cliquer sur la cellule, puis "executer") et indiquer ce qui se passe. 

In [ ]:
message = ('allume')
envoyer_message(message, comm_serie)

3. Quel code faut il écrire pour éteindre la del ?. Ecrire ce code dans la cellule ci-dessous et vérifier le fonctionnement

In [21]:
#écrire le code ici


4. Identifier le capteur et l'actionneur

## Activité 3 : commander l'éclairage (programme avancé)

Modifier un des  programmes précédents pour afficher à l'écran de l'ordinateur "obscurité" ou "éclairage normal" en fonction de la luminosité au niveau du capteur de lumière.

Tester le programme

In [ ]:
#écrire le code ici